# PCA dimensional reduction

Given the limited number of training samples (approximately 13k), it is advisable to decrease the number of features in order to reduce the total number of parameters required for a deep learning model. Therefore, to mitigate the "curse of dimensionality" while preserving the relevant information, PCA will be employed.

### **Code Functionality Breakdown**

#### **1. Purpose of the Code**
This code performs **PCA (Principal Component Analysis)** on different extracted audio features, such as acoustic indices, MPS, YAMNet, VGGish, and PANNs embeddings. It aims to reduce dimensionality, retain significant variance, and save transformed features for further analysis and modeling in your **speech emotion recognition project**.

---

### **2. Key Components**

#### **`PCAProcessor` Class**
The class encapsulates the workflow for:
- **Loading Features:** Loads the extracted features from disk.
- **PCA Transformation:** Reduces dimensionality of each feature set while preserving the desired variance.
- **Visualization:** Generates plots for variance explained and feature correlations.
- **Saving Results:** Saves transformed features and PCA models.

---

#### **2.1 Initialization**
- **`data_dir`:** Base directory containing the dataset and feature files.
- **`processed_dir`:** Directory where PCA models and transformed data are stored.
- **`visualization_dir`:** Directory for saving visualizations of PCA results.
- **`feature_configs`:** Configurations for each feature set, including:
  - **Name:** Human-readable identifier for the feature.
  - **Number of Components (`n_components`):** Maximum PCA components to retain.
  - **`handle_inf`:** Whether to handle infinities by replacing them with NaNs.
  - **`min_explained_variance`:** Minimum cumulative variance required (e.g., 95%).

---

#### **2.2 Feature Loading**
- **`load_data`:**
  - Loads the metadata (`train_val_test_split_EMODB.csv`) and feature files (e.g., `yamnet_embedding.npy`, `indices_raw.csv`).
  - Checks that all feature files exist and have the correct length.

- **`_validate_feature_lengths`:**
  - Ensures that the length of each feature set matches the number of samples in the dataset.

---

#### **2.3 PCA Transformation**
- **`get_transformer`:**
  - Creates a **PCA pipeline** with an imputer (to handle NaNs), PCA transformer, and a scaler.

- **`analyze_variance_explained`:**
  - Visualizes the explained variance for PCA components:
    - **Individual Variance:** Variance explained by each component.
    - **Cumulative Variance:** Total variance explained by the top `n` components.

- **`transform_feature`:**
  - Transforms a single feature set:
    1. **Handles NaNs/Infinities:** Replaces missing or infinite values with the mean.
    2. **Fits PCA on Training Data:** Retains only the required number of components.
    3. **Transforms All Splits:** Applies the fitted PCA model to training, validation, and test sets.
    4. **Saves PCA Model:** Saves the PCA transformer for later use.
    5. **Variance Analysis:** Generates plots for variance explained.

---

#### **2.4 Correlation Analysis**
- **`visualize_correlations`:**
  - Analyzes correlations between transformed feature sets for a given split (e.g., training data).
  - Creates a **heatmap** of mean absolute correlations between feature sets, providing insights into redundancy or complementarity.

---

#### **2.5 Feature Processing**
- **`process_features`:**
  - End-to-end workflow for processing all feature sets:
    1. **Load Data:** Loads dataset and feature files.
    2. **Define Splits:** Creates masks for training, validation, and test sets based on the fold column.
    3. **Transform Features:** Applies PCA transformation to each feature set.
    4. **Visualize Correlations:** Generates a heatmap of feature correlations.
    5. **Save Results:** Saves the aggregated transformed data and labels as a `.pkl` file.

---

### **3. Main Workflow**

#### **`main` Function**
- **Purpose:** Defines the entry point for running the PCA processing workflow.
- **Steps:**
  1. Initializes the `PCAProcessor` with paths and parameters.
  2. Calls `process_features` to process all feature sets.
  3. Logs the completion or errors.

---

### **4. Role in Your Project**

This code contributes significantly to your **speech emotion recognition project** by:
1. **Dimensionality Reduction:**
   - Reduces the size of feature sets (e.g., embeddings) while preserving essential information.
   - Helps avoid overfitting and reduces computational costs during modeling.

2. **Explained Variance Analysis:**
   - Provides insights into how many components are needed to retain a specific amount of variance.
   - Ensures an optimal balance between dimensionality and information retention.

3. **Correlation Analysis:**
   - Identifies redundancies or complementarities between feature sets.
   - Guides feature selection or engineering decisions.

4. **Standardized Data:** 
   - Prepares uniformly transformed feature sets for downstream modeling.

---

### **5. Outputs**
1. **Transformed Features:** Saved as `.pkl` or `.npy` files.
2. **PCA Models:** Saved as `.pkl` for reuse.
3. **Visualizations:**
   - **Variance Explained:** For each feature set.
   - **Feature Correlations:** Heatmaps showing relationships between feature sets.

---

Let me know if you’d like additional explanations, optimizations, or extensions to this code!

In [3]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from typing import Dict, Tuple, List, Any, Optional
import logging
from pathlib import Path
from dataclasses import dataclass
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('pca_processing.log'),
        logging.StreamHandler()
    ]
)

@dataclass
class FeatureConfig:
    """Configuration for PCA transformation of different features"""
    name: str
    n_components: int
    handle_inf: bool = False
    min_explained_variance: float = 0.95

class PCAProcessor:
    """Class to handle PCA transformations of audio features"""
    
    def __init__(self, 
                 data_dir: str, 
                 random_state: int = 23,
                 visualization_dir: Optional[str] = None):
        """
        Initialize PCA processor
        
        Args:
            data_dir: Base directory for data
            random_state: Random seed for reproducibility
            visualization_dir: Directory to save visualizations
        """
        self.data_dir = Path(data_dir)
        self.processed_dir = self.data_dir / 'processed'
        self.pca_dir = self.processed_dir / 'PCA_transformer'
        self.visualization_dir = Path(visualization_dir) if visualization_dir else self.processed_dir / 'visualizations'
        self.random_state = random_state
        
        # Create necessary directories
        self.pca_dir.mkdir(parents=True, exist_ok=True)
        self.visualization_dir.mkdir(parents=True, exist_ok=True)
        
        # Define feature configurations
        self.feature_configs = {
            'indices_raw': FeatureConfig('Acoustic Indices', 20, True, 0.95),
            'mps': FeatureConfig('MPS Features', 10, False, 0.9),
            'yamnet': FeatureConfig('YAMNet', 100, False, 0.95),
            'vggish': FeatureConfig('VGGish', 64, False, 0.95),
            'panns_clip': FeatureConfig('PANNs Clip', 50, False, 0.95),
            'panns_embedding': FeatureConfig('PANNs Embedding', 100, False, 0.95)
        }
        
    def load_data(self) -> Tuple[pd.DataFrame, Dict[str, np.ndarray]]:
        """Load all required data files with validation"""
        try:
            # Load dataset splits
            df_all = pd.read_csv(self.data_dir / 'train_val_test_split_EMODB.csv')
            logging.info(f"Loaded dataset with {len(df_all)} samples")
            
            # Define feature files
            feature_files = {
                'indices_raw': self.processed_dir / 'embeddings' / 'indices_raw.csv',
                'mps': self.processed_dir / 'embeddings' / 'mps_embedding.npy',
                'yamnet': self.processed_dir / 'embeddings' / 'yamnet_embedding.npy',
                'vggish': self.processed_dir / 'embeddings' / 'vggish_embedding.npy',
                'panns_clip': self.processed_dir / 'embeddings' / 'panns_clip.npy',
                'panns_embedding': self.processed_dir / 'embeddings' / 'panns_embedding.npy'
            }
            
            # Load features
            features = {}
            missing_files = []
            
            for feature_name, file_path in feature_files.items():
                if not file_path.exists():
                    missing_files.append(str(file_path))
                    continue
                    
                try:
                    if str(file_path).endswith('.csv'):
                        features[feature_name] = pd.read_csv(file_path)
                    else:
                        features[feature_name] = np.load(file_path)
                        
                    logging.info(f"Loaded {feature_name} with shape {features[feature_name].shape}")
                    
                except Exception as e:
                    logging.error(f"Error loading {feature_name}: {str(e)}")
                    missing_files.append(str(file_path))
            
            if missing_files:
                logging.warning(f"Missing feature files: {missing_files}")
                
            # Validate data lengths
            self._validate_feature_lengths(features, len(df_all))
            
            return df_all, features
            
        except Exception as e:
            logging.error(f"Error loading data: {str(e)}")
            raise
            
    def _validate_feature_lengths(self, features: Dict[str, np.ndarray], expected_length: int) -> None:
        """Validate that all features have the correct length"""
        for name, feature in features.items():
            actual_length = len(feature) if isinstance(feature, pd.DataFrame) else feature.shape[0]
            if actual_length != expected_length:
                raise ValueError(f"Feature {name} has length {actual_length}, expected {expected_length}")
                
    def get_transformer(self, n_components: int) -> Pipeline:
        """Create PCA transformer pipeline"""
        return Pipeline([
            ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
            ('pca', PCA(n_components=n_components, random_state=self.random_state)),
            ('scaler', StandardScaler())
        ])
        
    def analyze_variance_explained(self, 
                                 transformer: Pipeline, 
                                 feature_name: str,
                                 min_explained_variance: float) -> None:
        """Analyze and visualize PCA explained variance"""
        pca = transformer.named_steps['pca']
        explained_variance_ratio = pca.explained_variance_ratio_
        cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
        
        # Plot explained variance
        plt.figure(figsize=(12, 6))
        
        # Individual variance
        plt.subplot(1, 2, 1)
        plt.plot(explained_variance_ratio, 'b-', label='Individual')
        plt.xlabel('Principal Components')
        plt.ylabel('Explained Variance Ratio')
        plt.title(f'{feature_name} - Individual Explained Variance')
        plt.grid(True)
        
        # Cumulative variance
        plt.subplot(1, 2, 2)
        plt.plot(cumulative_variance_ratio, 'r-', label='Cumulative')
        plt.axhline(y=min_explained_variance, color='g', linestyle='--', 
                   label=f'{min_explained_variance*100}% Threshold')
        plt.xlabel('Number of Components')
        plt.ylabel('Cumulative Explained Variance Ratio')
        plt.title(f'{feature_name} - Cumulative Explained Variance')
        plt.grid(True)
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(self.visualization_dir / f'variance_explained_{feature_name}.png')
        plt.close()
        
        # Log variance information
        n_components_95 = np.argmax(cumulative_variance_ratio >= min_explained_variance) + 1
        logging.info(f"{feature_name}: {n_components_95} components needed for "
                    f"{min_explained_variance*100}% explained variance")
        
    def transform_feature(self, 
                         data: np.ndarray,
                         splits: Dict[str, np.ndarray],
                         config: FeatureConfig) -> Dict[str, np.ndarray]:
        """Transform a single feature set"""
        try:
            # Handle infinities if needed
            if config.handle_inf:
                if isinstance(data, pd.DataFrame):
                    data = data.replace([np.inf, -np.inf], np.nan).values
                else:
                    data = np.nan_to_num(data, nan=np.nan, posinf=np.nan, neginf=np.nan)
            
            # Get transformer and fit on training data
            transformer = self.get_transformer(config.n_components)
            transformer.fit(data[splits['train']])
            
            # Transform all splits
            transformed = {
                split_name: transformer.transform(data[split_mask])
                for split_name, split_mask in splits.items()
            }
            
            # Save transformer
            with open(self.pca_dir / f'{config.name.lower()}_transformer.pkl', 'wb') as f:
                pickle.dump(transformer, f)
                
            # Analyze variance explained
            self.analyze_variance_explained(
                transformer, 
                config.name,
                config.min_explained_variance
            )
            
            return transformed
            
        except Exception as e:
            logging.error(f"Error transforming {config.name}: {str(e)}")
            raise
            
    def visualize_correlations(self, 
                             transformed_data: Dict[str, np.ndarray],
                             split: str = 'train') -> None:
        """Visualize correlations between different feature sets"""
        try:
            # Collect all transformed features for the specified split
            features_dict = {
                name.replace(f'{split}_', '').replace('_pca', ''): data
                for name, data in transformed_data.items()
                if name.startswith(f'{split}_') and name.endswith('_pca')
            }
            
            # Calculate mean correlations between feature sets
            n_features = len(features_dict)
            correlation_matrix = np.zeros((n_features, n_features))
            feature_names = list(features_dict.keys())
            
            for i, (name1, features1) in enumerate(features_dict.items()):
                for j, (name2, features2) in enumerate(features_dict.items()):
                    if i <= j:
                        # Calculate mean absolute correlation
                        corr = np.mean(np.abs(np.corrcoef(features1.T, features2.T)))
                        correlation_matrix[i, j] = corr
                        correlation_matrix[j, i] = corr
            
            # Plot correlation matrix
            plt.figure(figsize=(10, 8))
            sns.heatmap(correlation_matrix, 
                       xticklabels=feature_names,
                       yticklabels=feature_names,
                       cmap='coolwarm',
                       annot=True,
                       fmt='.2f')
            plt.title(f'Mean Feature Correlations ({split} split)')
            plt.tight_layout()
            plt.savefig(self.visualization_dir / f'feature_correlations_{split}.png')
            plt.close()
            
        except Exception as e:
            logging.error(f"Error visualizing correlations: {str(e)}")
            
    def process_features(self) -> Dict[str, Any]:
        """Process all features with PCA"""
        try:
            # Load data
            df_all, features = self.load_data()
            
            if not features:
                raise ValueError("No features available for processing")
            
            # Define splits
            splits = {
                'train': df_all['fold'].isin([2, 3, 4]),
                'valid': df_all['fold'] == 1,
                'test': df_all['fold'] == 0
            }
            
            # Transform available features
            transformed_data = {}
            for feature_name, config in tqdm(self.feature_configs.items(), 
                                          desc="Processing features"):
                if feature_name in features:
                    transformed = self.transform_feature(
                        features[feature_name],
                        splits,
                        config
                    )
                    
                    for split_name, data in transformed.items():
                        transformed_data[f'{split_name}_{feature_name}_pca'] = data
            
            # Add labels
            for split_name, split_mask in splits.items():
                transformed_data[f'y_{split_name}'] = df_all.loc[split_mask, 'emotion_label']
            
            # Visualize feature correlations
            self.visualize_correlations(transformed_data, 'train')
            
            # Save aggregated data
            save_path = self.processed_dir / 'aggregated_data.pkl'
            with open(save_path, 'wb') as f:
                pickle.dump(transformed_data, f)
                
            logging.info(f"Saved aggregated data to {save_path}")
            
            return transformed_data
            
        except Exception as e:
            logging.error(f"Error in feature processing: {str(e)}")
            raise

def main():
    """Main execution function"""
    try:
        data_dir = '/Users/huangjuhua/文档文稿/NYU/Time_Series/data'
        
        processor = PCAProcessor(
            data_dir=data_dir,
            random_state=23,
            visualization_dir=os.path.join(data_dir, 'processed', 'visualizations')
        )
        
        processor.process_features()
        logging.info("PCA processing completed successfully")
        
    except Exception as e:
        logging.error(f"Processing failed: {str(e)}")

if __name__ == "__main__":
    main()

2024-12-07 17:01:59,357 - INFO - Loaded dataset with 535 samples
2024-12-07 17:01:59,361 - INFO - Loaded indices_raw with shape (535, 33)
2024-12-07 17:01:59,368 - INFO - Loaded mps with shape (535, 8080)
2024-12-07 17:01:59,370 - INFO - Loaded yamnet with shape (535, 1024)
2024-12-07 17:01:59,372 - INFO - Loaded vggish with shape (535, 128)
2024-12-07 17:01:59,372 - WARNING - Missing feature files: ['/Users/huangjuhua/文档文稿/NYU/Time_Series/data/processed/embeddings/panns_clip.npy', '/Users/huangjuhua/文档文稿/NYU/Time_Series/data/processed/embeddings/panns_embedding.npy']
Processing features: 100%|██████████| 6/6 [00:07<00:00,  1.21s/it]
2024-12-07 17:02:06,816 - INFO - Saved aggregated data to /Users/huangjuhua/文档文稿/NYU/Time_Series/data/processed/aggregated_data.pkl
2024-12-07 17:02:06,818 - INFO - PCA processing completed successfully
